In [ ]:
# imports here

Distinguish pits by snow depth classes: 'shallow', 'medium', 'deep'

PLOT: #1 Density

  - depth (cm) vs. density (kg/m3) for snow depth classes
  - 3 colors: shallow (r), medium (g), deep (b)
  - vertical bars to show the 10cm measurement increments (see ../images)  
  - time period -- (hmm, all? single day? 1 week avg?)

PLOT: #2 Temperature

  - depth (cm) vs. temperature (C) for snow depth classes
  - 3 colors: shallow (r), medium (g), deep (b)
  - x-axis is -10 to 0 (see ../images)  
  - time period -- (hmm, all? single day? 1 week avg?)
  - could plot all pits with light gray line (or group by depth classes in a light/transparent color) and then take the avg. of 'shallow', 'medium', 'deep' and plot with a bold color line

PLOT: #3
  - do something with stratigraphy here....still thinking on that...

In [ ]:
from snowexsql.db import get_db
from snowexsql.data import SiteData, ImageData, LayerData, PointData
from snowexsql.conversions import query_to_geopandas
import datetime

def parse_class(site_id):
    # Classifying by expected depth 
    depth_class = {'shallow':[1, 3], 'medium':[4, 6], 'deep':[7, 9]}
   
    dclass = None 
    
    class_id = site_id[0]
    
    if class_id.isnumeric():
        class_id = int(class_id)

        for k,v in depth_class.items():

            if class_id >= v[0] and class_id <= v[1]:
                dclass = k 
                break

    return dclass 


# load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

result = session.query(LayerData.type).distinct().all()
print(result)

qry = session.query(LayerData).filter(LayerData.type=='density')

# Form our dataframe from the query 
df = query_to_geopandas(qry, engine)

# Form a handy lambda for performant ops in df's
depth_class_lambda = lambda row: parse_class(row['site_id'])
df['depth_class'] = df.apply(depth_class_lambda, axis=1)

# Show off our df 
df.plot()